 # Building Lighting for AHFC Headquarters building

In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import scrapbook as sb

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
building_id = 37
server_web_address = 'https://bms.ahfc.us'

In [ ]:
# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 4)
sb.glue('title', 'Lighting')

server = Server(server_web_address)

building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']

indoor_light_sensors = bmondata.csnl_to_list(server.buildings(building_id)[0]['indoor_lights'])

if len(indoor_light_sensors) == 0:
    error_message = 'This building does not appear to have any lighting data. Either there is no data source or it has not been properly configured.'
    raise RuntimeError(error_message)    

starting_timestamp = datetime.datetime.now() - relativedelta(weeks=1)

if server.sensor_readings(indoor_light_sensors,start_ts = starting_timestamp,
                          end_ts = datetime.datetime.now(),averaging='15min').empty:
    if server.sensor_readings(indoor_light_sensors,start_ts = starting_timestamp,
                              end_ts = datetime.datetime.now(),averaging='1H').empty:
        lighting_current_week = server.sensor_readings(indoor_light_sensors,start_ts = starting_timestamp,
                                                       end_ts = datetime.datetime.now(),averaging='2H')
    else:
        lighting_current_week = server.sensor_readings(indoor_light_sensors,start_ts = starting_timestamp,
                                                       end_ts = datetime.datetime.now(),averaging='1H')
else:
    lighting_current_week = server.sensor_readings(indoor_light_sensors,start_ts = starting_timestamp,
                                                   end_ts = datetime.datetime.now(),averaging='15min')

diverging_hue_list = ['#a6cee3',
                      '#1f78b4',
                      '#b2df8a',
                      '#33a02c',
                      '#fb9a99',
                      '#e31a1c',
                     '#fdbf6f']

# Get schedule and timezone from the API data
building_schedule = building_df[0]['schedule']
building_timezone = building_df[0]['timezone']

# Create a schedule object using Ian's library
schedule_object = bmondata.Schedule(building_schedule, building_timezone)

# Set the start and end times of the graph
graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
graph_end_date = datetime.datetime.now()

# Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

# Loop through the list of occupied timestamps and convert them to datetimes
start_time_list = []
end_time_list = []

for i in np.arange(0, len(list_of_occupied_timestamps)):
    start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
    start_time_list.append(start_datetime)
    end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
    end_time_list.append(end_datetime)


if building_schedule == '' or schedule_object is None:
    error_message = 'This building does not appear to have any defined occupied schedule. Please define the occupied hours through the administrator interface.'
    raise RuntimeError(error_message)
    
else:
    # Set the start and end times of the graph
    graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
    graph_end_date = datetime.datetime.now()

    # Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
    list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

    # Loop through the list of occupied timestamps and convert them to datetimes
    start_time_list = []
    end_time_list = []

    for i in np.arange(0, len(list_of_occupied_timestamps)):
        start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
        start_time_list.append(start_datetime)
        end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
        end_time_list.append(end_datetime)

    # Create variables to indicate the first occupied start time and the last occupied end time
    occ_start = start_time_list[0]
    occ_end = end_time_list[-1]
    occ_day_list = np.arange(occ_start.weekday(), occ_end.weekday()+1)
    
    min_value = lighting_current_week.min().min()
    max_value = lighting_current_week.max().max()
    
    shape_list = []
    shape_dict = {'type':'rect',
                  'fillcolor':'#bdbdbd',
                  'opacity':0.35,
                  'line': {'width':1},
                  'layer':'below',
                  'y0':min_value,
                  'y1':max_value
                 }
    
    if graph_start_date < start_time_list[0]:
        shape_dict.update(x0=graph_start_date)
        shape_dict.update(x1=start_time_list[0])
        shape_list.append(shape_dict.copy())
        
    for i in np.arange(len(start_time_list)):
        if start_time_list[-1] != start_time_list[i]:
            shape_dict.update(x0=end_time_list[i])
            shape_dict.update(x1=start_time_list[i+1])
            shape_list.append(shape_dict.copy())
            
    if graph_end_date > end_time_list[-1]:
        shape_dict.update(x0=end_time_list[-1])
        shape_dict.update(x1=graph_end_date)
        shape_list.append(shape_dict.copy())

title_md = '''# Building Lighting for {} building'''
title_md = title_md.format(current_building_name)

#####################################################################

Markdown(title_md)

counter = 1

data = []
for light_sensor in lighting_current_week.columns.values:
    sensor_title = server.sensors(light_sensor)[0]['title']
    
    light_sensor = go.Scatter(x=lighting_current_week.index,
                             y=lighting_current_week[light_sensor],
                             name=sensor_title,
                             line=dict(color=diverging_hue_list[(counter-1)]))
    counter += 1
    data.append(light_sensor)
    
layout = dict(title='Current Week Lighting Usage',
              xaxis=dict(title='Date and Time'),
              yaxis=dict(title='Light Levels (lux)'),
              shapes = shape_list,
              showlegend = True
             )


fig = go.Figure(dict(data=data, layout=layout))
fig.show()

## Grey areas indicate that the building is unoccupied; ensuring that lights are being turned off at night and on the weekends is a simple way to save energy and money. 